# Segmenting and Clustering Neighborhoods in Toronto

My coursera submission to segment and cluster neighborhoods.

<span style="font-size:xx-small;"><b>Do not copy or distribute.</b></span>

In [1]:
import pandas as pd
import numpy as np
import requests
import json

user_agent = 'datascience jupyter notebook/0.0 (Austin Rainwater, paco@heckin.io)'

## Step 2

 > Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

I will start by grabbing the data from Wikipedia using its API.

See https://stackoverflow.com/questions/40210536/how-to-obtain-data-in-a-table-from-wikipedia-api

In [2]:
url = 'https://en.wikipedia.org/w/api.php?action=parse&format=json&prop=sections&page=List_of_postal_codes_of_Canada%3A_M'

results = requests.get(url, headers={'User-Agent': user_agent}).json()
sections = results['parse']['sections']

section_index = None
for section in sections:
    if section['line'] == 'Toronto - 103 FSAs':
        section_index = section['index']

if section_index is None:
    raise ValueError("The Toronto FSAs were not found.")
    
"Section index is " + section_index

'Section index is 1'

In [3]:
url = 'https://en.wikipedia.org/w/api.php?action=parse&format=json&page=List_of_postal_codes_of_Canada%3A_M&prop=text&section=' + section_index

results = requests.get(url, headers={'User-Agent': user_agent}).json()
html = results['parse']['text']['*']

"Beginning of HTML: " + html[:1000]

'Beginning of HTML: <div class="mw-parser-output"><h2><span class="mw-headline" id="Toronto_-_103_FSAs"><a href="/wiki/Toronto" title="Toronto">Toronto</a> - 103 <a href="/wiki/Postal_codes_in_Canada#Forward_sortation_areas" title="Postal codes in Canada">FSAs</a></span><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=List_of_postal_codes_of_Canada:_M&amp;action=edit&amp;section=1" title="Edit section: Toronto - 103 FSAs">edit</a><span class="mw-editsection-bracket">]</span></span></h2>\n<p>Note: There are no rural FSAs in Toronto, hence no postal codes should start with M0. However, the postal code M0R 8T0 is assigned to an <a href="/wiki/Amazon_(company)" title="Amazon (company)">Amazon</a> warehouse in Mississauga, suggesting that Canada Post may have reserved the M0 FSA for high volume addresses.\n</p>\n<table class="wikitable sortable">\n<tbody><tr>\n<th>Postal Code\n</th>\n<th>Borough\n</th>\n<th>Neighbourhood\n</th></tr>\n<tr>

I was going to try to build a [regular expression to parse the HTML](https://stackoverflow.com/a/1732454), but then I learned that pandas already has a [`read_html()` function](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html):

In [4]:
df = pd.read_html(html)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Step 3

 > - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
 > - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
 > - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
 > - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
 > - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 > - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


I'll grab the values that are not "`Not assigned`", and then get the rest of the unique values to see what's left.

In [5]:
df = df[df['Borough'] != 'Not assigned']
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

So, looking at the page, it looks like it's been recently edited so that the format is actually as stated in the instructions. I can confirm this by seeing how many unique boroughs each postal code has, as well as how many unique neighborhood values are in each borough.

In [6]:
a = df[['Postal Code', 'Borough']].groupby('Postal Code').nunique().max()
b = df.groupby(['Postal Code', 'Borough']).nunique().max()
print(f"Max unique boroughs per postal code: {int(a)}")
print(f"Max unique neighborhoods per borough: {int(b)}")

Max unique boroughs per postal code: 1
Max unique neighborhoods per borough: 1


However, I am curious if I can list out the neighborhoods in each postal code.

In [7]:
grouped_df = df[['Postal Code', 'Neighbourhood']].groupby('Postal Code')#.transform(', '.join)
grouped_df.aggregate(', '.join)

,Neighbourhood
Postal Code,
M1B,"Malvern, Rouge"
M1C,"Rouge Hill, Port Union, Highland Creek"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae
...,...
M9N,Weston
M9P,Westmount
M9R,"Kingsview Village, St. Phillips, Martin Grove ..."


I would use the same concept to perform the aggregation as requested on boroughs.

Now to continue assigning a value to "not assigned" neighborhoods.

In [8]:
def process_row(row):
    if row['Neighbourhood'] == 'Not assigned':
        return row['Borough']
    return row['Neighbourhood']

df['Neighbourhood'] = df.apply(process_row, axis=1)
'Not assigned' in df['Neighbourhood']

False

Let's finally check the shape of the dataframe.

In [9]:
df.shape

(103, 3)